In [29]:
import pandas as pd 

training = pd.read_csv('../Data/training.csv', usecols=["emotion", "image"," pixels"])
validation = pd.read_csv('../Data/validation.csv', usecols=["emotion", "image"," pixels"])
test = pd.read_csv('../Data/test.csv', usecols=["emotion", "image"," pixels"])

In [35]:
import numpy as np
from matplotlib import image
from skimage.feature import hog

"""
Transforms the imgs in the training/testing/validation dataset and saves them as npy

Arguments:
dataset: 'train', 'test', or 'validate'
num_imgs: number of imgs/pixels in the csv dataset

"""

def resize_and_flip(dataset, num_imgs):
    hog_features = []
    emotion_classes = []
    
    if dataset=='test':
        df = test
    elif dataset=='validate':
        df = validation
    elif dataset=='train':
        df = training
    else:
        print("ERR: dataset input is incorrect")
        return
    
    for i in range(0, num_imgs):
        img_pixels = df.iloc[i][" pixels"]
        
        # if images are jpg 
        if type(img_pixels) is float:
            imgpath = '../Data/MuxspaceDataset/images/'
            img_pixels = image.imread(imgpath + str(df.iloc[i]["image"]))   # load image as pixel array
            img_data = np.asarray(np.resize(img_pixels, (1,48,48)), dtype=np.float64)
            img_data_flipped = np.fliplr(img_data)
           
        # if images are pixels
        else:
            img_pixels = df.iloc[i][" pixels"]
            img_string = img_pixels.split(' ')
            img_data = np.asarray(img_string, dtype=np.float64).reshape(1,48, 48)
            img_data_flipped = np.fliplr(img_data)
        
        features, _ = hog(img_data, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=True)
        hog_features.append(features)
        emotion_classes.append(df.iloc[i]["emotion"])
        
        features, _ = hog(img_data_flipped, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=True)
        hog_features.append(features)
        emotion_classes.append(df.iloc[i]["emotion"])
    
    master_path = './ProcessedData/' + str(dataset)
    np.save(master_path + '/hog_features.npy', hog_features)
    np.save(master_path + '/emotion_classes.npy', emotion_classes)

In [ ]:
resize_and_flip(dataset='train', num_imgs=3696)
resize_and_flip(dataset='validate', num_imgs=791)
resize_and_flip(dataset='test', num_imgs=798)